In [ ]:
%tensorflow_version 1.x
!nvidia-smi

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

nb_classes = 10
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()
# train_x, test_x = train_x / 255.0, test_x / 255.0

train_mean = train_x.mean(axis=0)
train_std = train_x.std(axis=0)

def normalize_X(x, train_mean, train_std):
  return (x - train_mean) / train_std
train_x = normalize_X(train_x, train_mean, train_std)
test_x = normalize_X(test_x, train_mean, train_std)

train_y = tf.keras.utils.to_categorical(train_y, nb_classes)
test_y = tf.keras.utils.to_categorical(test_y, nb_classes)

## MobilenetV2-like network without Class

In [ ]:
def mbconv_block(inputs, in_c, out_c, k, e, s):
    """
    in_c : channels of input tensor of this block
    out_c : channels of output tensor of this block
    k : kernel size of depthwise conv
    e : expand ratio
    s : strides
    """
    mid_c = in_c * e

    x = keras.Sequential([
      layers.Conv2D(filters=mid_c, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.DepthwiseConv2D(kernel_size=k, strides=s, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.Conv2D(filters=out_c, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
    ])(inputs)

    has_shortcut = (in_c == out_c) and s == 1
    if has_shortcut:
      shortcut = inputs
      x = x + shortcut
    
    return x

def mobilenetv2(inputs):
    # stem part
    first_layers = keras.Sequential([
      layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.DepthwiseConv2D(kernel_size=3, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.Conv2D(filters=16, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
    ])
    x = first_layers(inputs)
  
    # MBConvs
    x = mbconv_block(x, 16, 24, k=3, e=3, s=1)
    x = mbconv_block(x, 24, 24, k=3, e=3, s=1)
  
    x = mbconv_block(x, 24, 32, k=3, e=3, s=2)
    x = mbconv_block(x, 32, 32, k=3, e=3, s=1)
    x = mbconv_block(x, 32, 32, k=3, e=3, s=1)
  
    x = mbconv_block(x, 32, 64, k=3, e=3, s=2)
    x = mbconv_block(x, 64, 64, k=3, e=3, s=1)
    x = mbconv_block(x, 64, 64, k=3, e=3, s=1)
  
    # head part
    last_layers = keras.Sequential([
      layers.Conv2D(filters=128, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dropout(rate=0.2),
      layers.Dense(10),
    ])
    output = last_layers(x)
  
    return output

In [ ]:
# build keras.Model
inputs = keras.Input(shape=(32, 32, 3))
outputs = mobilenetv2(inputs)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epoch = 15
batch_size = 256
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))

## Class-based MobileNetV2 CIFAR10

In [ ]:
class MBConvBlock(keras.layers.Layer):
  def __init__(self, in_c, out_c, k, e, s):
    super().__init__()
    self.has_shortcut = (in_c == out_c) and s == 1
    
    mid_c = in_c * e
    self.residual = keras.Sequential([
      layers.Conv2D(filters=mid_c, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.DepthwiseConv2D(kernel_size=k, strides=s, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.Conv2D(filters=out_c, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
    ])

  def call(self, inputs):
    x = self.residual(inputs)
    if self.has_shortcut:
      x = x + inputs
    
    return x

class MobileNetV2(keras.Model):
  def __init__(self, input_shape=None):
    super().__init__()
    self._input_shape = input_shape
  
  def build(self, input_shape): # automatically called when first __call__ invokes.
    # stem layers
    self.first_layer = keras.Sequential([
      layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.DepthwiseConv2D(kernel_size=3, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.Conv2D(filters=16, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
    ])

    # MBConvBlocks
    self.blocks = []
    
    in_c_per_stage = [16, 24, 32]
    out_c_per_stage = [24, 32, 64]
    strides_of_first_block_per_stage = [1, 2, 2]
    repeats_per_stage = [2, 3, 3]

    kernel_size = 3
    expand_ratio = 3
    for in_c, out_c, strides, repeats in zip(in_c_per_stage, out_c_per_stage, strides_of_first_block_per_stage, repeats_per_stage):
      for _ in range(repeats):
        self.blocks.append(MBConvBlock(in_c, out_c, kernel_size, expand_ratio, strides))
        in_c = out_c
        strides = 1

    # head layers
    self.last_layer = keras.Sequential([
      layers.Conv2D(filters=128, kernel_size=1, strides=1, padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(6),
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dropout(rate=0.2),
      layers.Dense(10),
    ])

  def call(self, inputs): # automatically called in __call__ after build().
    x = self.first_layer(inputs)
    for block in self.blocks:
      x = block(x)
    x = self.last_layer(x)

    return x

In [ ]:
model = MobileNetV2()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

epoch = 15
batch_size = 256

model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))